# Master the linear fit 
## uncertainties, interpolation and extrapolation
More Coffee, More Confidence [ESO Chile 26/05/2016 - Antoine Mérand]

For the vast majority of the numerical problems we encounter as astronomers consist of comparing a model to some data, and adjust the parameters of the model to the data. 

## Context

Fitting a straight line is the most basic data fitting everybody should master. The widest used technique is to minimize the L2 distance between the data points (Xi,Yi) and the model f(x) = a.x + b:

$$ L_2 = \sum_i \left| Y_i - aX_i - b\right|^2$$ 

This is also called a $\chi^2$. Note that the minimum has an analytical solution:
$$ \partial \chi^2 / \partial a = 0 , \partial \chi^2 / \partial b = 0 $$
i.e.
$$ \partial \chi^2 / \partial a = \sum_i -2\times X_i\left( Y_i - a\times X_i - b\right) = 0 $$
$$ \partial \chi^2 / \partial b = \sum_i -2\times \left( Y_i - a\times X_i - b\right) = 0 $$
which corresponds to the following system of linear equations:
$$   a\sum_i X_i^2 + b \sum_iX_i = \sum_i  X_iY_i $$
$$   a\sum_iX_i + b = \sum_i  Y_i $$

$$ \begin{bmatrix} 
\sum_i X_i^2 &  \sum_iX_i\\ 
\sum_iX_i &  1 \\  
\end{bmatrix} 
\times 
\begin{bmatrix} 
a \\ 
b \\
\end{bmatrix} 
=  
\begin{bmatrix}
\sum_i  X_iY_i \\
\sum_i  Y_i \\
\end{bmatrix}
$$


## A simple example

### First we generate some data points with noise

In [ ]:
%pylab inline

In [ ]:
N = 10
a, b = 1.1, 2.2
# -- abscissa X between 1 and 2
X = 1+np.random.rand(N) 
Y = a*X + b 
# -- adding gaussian noise
E = 0.2*np.ones(N)
Y += E*np.random.randn(N)

### We plot our points and the simple analytical polynomial fit
np.polyfit provides a simple yet complete implementation of the polynomial fit.

In [ ]:
plt.errorbar(X, Y, yerr=E, marker='o', color='k', label='data points', linestyle='')
c = np.polyfit(X, Y, 1)
_x = np.linspace(1,2,10); plt.xlim(_x.min(), _x.max())
print 'a=', a, '->', c[0]
print 'b=', b, '->', c[1]
plt.plot(_x, np.polyval(c,_x), '-r', linewidth=2, label='fit')
plt.plot(_x, np.polyval([a,b],_x), '--r', linewidth=2, label='true')
plt.legend(loc='upper left')

## Incertainties on 'a' and 'b'?
The statistical uncertainties can be computed using the 2nd order derivatives of the $\chi^2$. Assuming that the data have a Gaussian (and uncorrelated) noise, the errors are accessible in the covariance matrix from np.polyfit.

In [ ]:
c, cov = np.polyfit(X, Y, 1, cov=True)
print 'a =', a, '->', c[0], '+/-', np.sqrt(cov[0,0])
print 'b =', b, '->', c[1], '+/-', np.sqrt(cov[1,1])
plt.plot(a, b, 'or')
plt.errorbar(c[0], c[1], xerr=np.sqrt(cov[0,0]), yerr=np.sqrt(cov[1,1]))
print cov

## Extrapolating / Interpolating

- You **interpolate** Y to values **within** the range of the data X_i
- You **extrapolate** Y to values **outside** the range of the data X_i

In both cases, one should attribute an uncertainty to the extrapolated / interpolated value...

In other words, **how to compute this uncertainty from the parameters (a,b) value and uncertainties?**

In [ ]:
plt.plot(X, Y, 'ok') # original data points
_x = linspace(0.5,2.5,100)
for i in range(500):
    p = [c[0]+np.random.randn()*np.sqrt(cov[0,0]),
         c[1]+np.random.randn()*np.sqrt(cov[1,1])]
    plt.plot(_x, np.polyval(p, _x), '-r', alpha=0.02)
plt.plot(_x, np.polyval(c,_x), '-r', linewidth=2, label='fit')
plt.plot(_x, np.polyval([a,b],_x), '--r', linewidth=2, label='true')
plt.legend(loc='upper left')

Gut Feelings:
- uncertainty on interpolation should be of the order of data scatter
- uncertainty on extrapolation should increase as we go further away from the original data set




**What is happening??? **

### Fitted parameters 'a' and 'b' are correlated
The correlation factor is the non-diagonal term on the covariance matrix

In [ ]:
print 'correlation a, b =', cov[1,0]/np.sqrt(cov[0,0]*cov[1,1])

In [ ]:
tmp = np.random.multivariate_normal(c, cov, 1000)
plt.plot(tmp[:,0], tmp[:,1], '.k', alpha=0.2)
plt.plot(a, b, 'or')
plt.errorbar(c[0], c[1], xerr=np.sqrt(cov[0,0]), yerr=np.sqrt(cov[1,1]), linewidth=2)

In [ ]:
plt.plot(X, Y, 'ok') # original data points
_x = linspace(0.5,2.5,100)
for i in range(500):
    p = np.random.multivariate_normal(c, cov)
    plt.plot(_x, np.polyval(p, _x), '-r', alpha=0.015)
plt.plot(_x, np.polyval(c,_x), '-r', linewidth=2, label='fit')
plt.plot(_x, np.polyval([a,b],_x), '--r', linewidth=2, label='true')
plt.legend(loc='upper left')

## How do I get an uncorrelated parametrization?
In the case of a linear fit, use mean-centered data. In the case of the linear fit, it removes the non-diagonal terms in the linear system: 
$$ \begin{bmatrix} 
\sum_i X_i^2 &  \sum_iX_i=0\\ 
\sum_iX_i=0 &  1 \\  
\end{bmatrix} 
\times 
\begin{bmatrix} 
a \\ 
b \\
\end{bmatrix} 
=  
\begin{bmatrix}
\sum_i  X_iY_i \\
\sum_i  Y_i \\
\end{bmatrix}
$$
then $a = \frac{1}{\sum_i X_i^2}\sum_i  X_iY_i $ and $b=\sum_i  Y_i$. b is simply the average of Y and a is the correlation between X and Y, normalized to the autocorrelation of the X. a and b are uncorrelated because if one shift all Y by a certain value c, it will shift b not not a:
$$ \frac{1}{\sum_i X_i^2}\sum_i  X_i(Y_i + c) = \frac{1}{\sum_i X_i^2}c\sum_i  X_i + \frac{1}{\sum_i X_i^2}\sum_i  X_iY_i = \frac{1}{\sum_i X_i^2}\sum_i  X_iY_i$$

In [ ]:
print '-- Naive approch (correlated):'
print '  Y = a*X + b'
c, cov = np.polyfit(X, Y, 1, cov=True)
print ' a =', a, '->', c[0], '+/-', np.sqrt(cov[0,0])
print ' b =', b, '->', c[1], '+/-', np.sqrt(cov[1,1])
print ' correlation a, b =', cov[1,0]/np.sqrt(cov[0,0]*cov[1,1])
print '-- Mean-centered data set (uncorrelated):'
print '  Y = a*(X-%f) + b'%1.5
c, cov = np.polyfit(X-1.5, Y, 1, cov=True)
print ' a ->', c[0], '+/-', np.sqrt(cov[0,0])
print ' b ->', c[1], '+/-', np.sqrt(cov[1,1])
print ' correlation a, b =', cov[1,0]/np.sqrt(cov[0,0]*cov[1,1])

The above parametrization is superior:
- user friendly for people who want to use your model
- actually more meaningful: **the zero-point is much better constrained**

In [ ]:
plt.plot(X, Y, 'ok') # original data points
_x = linspace(0.5,2.5,100)
for i in range(500):
    p = [c[0]+np.random.randn()*np.sqrt(cov[0,0]),
         c[1]+np.random.randn()*np.sqrt(cov[1,1])]
    plt.plot(_x, np.polyval(p, _x-X.mean()), '-r', alpha=0.015)
plt.plot(_x, np.polyval(c,_x-X.mean()), '-r', linewidth=2, label='fit')
plt.plot(_x, np.polyval([a,b],_x), '--r', linewidth=2, label='true')
plt.legend(loc='upper left')